In [105]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import (
    ensemble,
    preprocessing,
    tree,
)

from sklearn.metrics import (
    auc,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)

from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
)

from yellowbrick.classifier import (
    ConfusionMatrix,
    ROCAUC,
)

from yellowbrick.model_selection import (
    LearningCurve,
)

from pathlib import Path

In [106]:
ROOT_FILE = Path.cwd().parent
DATA_FILE = ROOT_FILE / "datasets" / "titanic3.csv"

Yellow Brick é uma biblioteca de visualização e diagnóstico para aprendizado de máquina que visa capacitar os usuários a visualizar os processos internos de seus modelos de aprendizado de máquina e permitir que eles tomem decisões mais informadas durante o processo de modelagem, facilitando a compreensão das forças e fraquezas de seus modelos.

## Faça uma pergunta

Essa corresponde a primeira etapa do processo de machine learning, onde você deve se perguntar o que deseja descobrir com os dados que possui. Para o exemplo do livro, será criado um modelo preditivo para responder uma pergunta: O indivíduo sobreviveu ou não ao naufrágio do Titanic? Terá-se como base os dados de passageiros do navio, como idade, sexo, classe social, etc.

É uma pergunta de classificação, pois estamos fazendo a predição de um rotulo, que é a sobrevivência ou não do passageiro. Se fosse uma pergunta de regressão, seria algo como: Qual a idade do passageiro?

## Colete os dados

A segunda etapa é a coleta dos dados, que pode ser feita de diversas formas, como por exemplo, através de um web crawler, que é um programa que navega pela web e coleta os dados de interesse. Pode ser feita pelo download de um dataset, como o do Kaggle, ou até mesmo através de uma API.

Para o exemplo do livro, estou mostrando duas maneiras, uma através de uma URL e outra através do download de um dataset.

In [107]:
url = ("https://hbiostat.org/data/repo/titanic3.csv")
df = pd.read_csv(url)

## OU

df = pd.read_csv(DATA_FILE)

df_orig = pd.read_csv(DATA_FILE)


Tem-se os seguintes dados:

- **pclass**: Classe do passageiro (1 = primeira classe; 2 = segunda classe; 3 = terceira classe)

- **survived**: Sobreviveu ou não (0 = Não; 1 = Sim)

- **name**: Nome do passageiro

- **sex**: Sexo do passageiro

- **age**: Idade do passageiro

- **sibsp**: Número de irmãos e cônjuges a bordo

- **parch**: Número de pais e filhos a bordo

- **ticket**: Número do ticket

- **fare**: Tarifa do passageiro

- **cabin**: Número da cabine

- **embarked**: Porto de embarque (C = Cherbourg; Q = Queenstown; S = Southampton)

- **boat**: Bote salva-vidas

- **body**: Número de identificação do corpo

- **home.dest**: Destino

## Limpe os dados

A maioria dos modelo do scikit-learn exige que os dados sejam numéricos. Os modelos podem falhar caso recebam valores ausentes. Além disso alguns modelos podem ter melhores desempenhos se os dados estiverem padronizados.

Podem também existir os chamados leaky features(dados que vazam informações). Essas variaveis contém informações sobre o futuro ou o resultado que se deseja prever

In [108]:
df.dtypes # Verificando os tipos de dados

pclass         int64
survived       int64
name          object
sex           object
age          float64
sibsp          int64
parch          int64
ticket        object
fare         float64
cabin         object
embarked      object
boat          object
body         float64
home.dest     object
dtype: object

Os tipos de dados mais comuns são:

- **int64**: Números inteiros

- **float64**: Números reais

- **datetime64[ns]**: Data e hora

- **object**: String, mas pode ser uma combinação entre strings e outros tipos

No geral tipos inteiros não apresentam problemas. Tipos float podem apresentar problemas com valores ausentes. Tipos data e string deverão ser convertidos para números.

Tipos string com baixa cardinalidade (poucos valores únicos) são chamados de colunas de categoria, é possível gerar colunas dummy (binárias) para cada valor único

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


Temos 6 variaveis númericas, 7 variaveis categóricas e 1 variavel booleana

In [110]:
df.shape # Verificando o tamanho do dataset

(1309, 14)

In [111]:
df.describe() # Verificando as estatísticas do dataset

,pclass,survived,age,sibsp,parch,fare,body
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,2.294882,0.381971,29.881138,0.498854,0.385027,33.295479,160.809917
std,0.837836,0.486055,14.413493,1.041658,0.865560,51.758668,97.696922
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200,328.000000


In [112]:
df.describe().iloc[:, :2] # iloc é um método para selecionar 
# linhas e colunas por números inteiros. Neste caso, estamos
# selecionando todas as linhas e as duas primeiras colunas

,pclass,survived
count,1309.000000,1309.000000
mean,2.294882,0.381971
std,0.837836,0.486055
min,1.000000,0.000000
25%,2.000000,0.000000
50%,3.000000,0.000000
75%,3.000000,1.000000
max,3.000000,1.000000


- **count:**  quantidade de valores não nulos

- **mean:** média dos valores

- **std:** desvio padrão

- **min:** valor mínimo

- **25%:** primeiro quartil

- **50%:** segundo quartil (mediana)

- **75%:** terceiro quartil

- **max:** valor máximo

Count é interessa para identificar valores ausentes. A mediana é mais robusta que a média, pois não é afetada por outliers.

Min e máx são importantes para identificar outliers. O desvio padrão é uma medida de dispersão, quanto maior o desvio padrão, maior a dispersão dos dados.

```python

df.isnull() # retorna um dataframe com valores booleanos, onde True indica que o valor é nulo

df.isnull().sum() # retorna a soma dos valores nulos de cada coluna

```

In [113]:
df.isnull().sum() # Verificando a quantidade de valores nulos

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

```python

df.isnull().mean() # Verificando a porcentagem de valores nulos

```

Útil para identificar colunas com muitos valores nulos

In [114]:
df.isnull().sum(axis=1).loc[:10] # Verificando a quantidade de valores nulos por linha

0     1
1     1
2     2
3     1
4     2
5     1
6     1
7     2
8     1
9     2
10    1
dtype: int64

A coluna body deve ser removida pois contém muitos valores nulos e pode ser considerada uma leaky feature (informa que o passageiro morreu).

A coluna boat também deve ser removida pois contém muitos valores nulos e pode ser considerada uma leaky feature (informa que o passageiro sobreviveu).


In [115]:
mask = df.isnull().any(axis = 1) # Criando uma máscara para selecionar as linhas com valores nulos
# any é um método que retorna True se qualquer elemento do eixo for True
mask.head()

0    True
1    True
2    True
3    True
4    True
dtype: bool

In [116]:
df[mask].body.head()

0      NaN
1      NaN
2      NaN
3    135.0
4      NaN
Name: body, dtype: float64

In [117]:
df.sex.value_counts() # Verificando a quantidade de valores por categoria

sex
male      843
female    466
Name: count, dtype: int64

In [118]:
df.embarked.value_counts(dropna=False) # Verificando a quantidade de valores por categoria, incluindo os valores nulos

embarked
S      914
C      270
Q      123
NaN      2
Name: count, dtype: int64

Temos 2 valores nulos na coluna embarked, podemos substituir pela moda (valor mais frequente), remover a linha, ou criar um dummy, ou então substituir pela média.

#### Criando os atributos

In [119]:
name = df['name']
# ou
name = df.name
name.head(3)

0     Allen, Miss. Elisabeth Walton
1    Allison, Master. Hudson Trevor
2      Allison, Miss. Helen Loraine
Name: name, dtype: object

In [120]:
df = df.drop(
    columns=[
        'name',
        'ticket',
        'home.dest',
        'boat',
        'body',
        'cabin',
    ]
)

Utiliza-se o método `drop` para remover linhas ou colunas, o padrão é remover linhas, para remover colunas, deve-se passar o parâmetro `axis=1`.

Remove-se, então: name, ticket, cabin, boat, body e home.dest

boat e body são leaky features, pois informam se o passageiro sobreviveu ou não

name, ticket e cabin são colunas de alta cardinalidade, ou seja, possuem muitos valores únicos, o que pode dificultar o aprendizado do modelo

home.dest é uma coluna de baixa cardinalidade, mas não é relevante para o modelo

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int64  
 1   survived  1309 non-null   int64  
 2   sex       1309 non-null   object 
 3   age       1046 non-null   float64
 4   sibsp     1309 non-null   int64  
 5   parch     1309 non-null   int64  
 6   fare      1308 non-null   float64
 7   embarked  1307 non-null   object 
dtypes: float64(2), int64(4), object(2)
memory usage: 81.9+ KB


In [122]:
df = pd.get_dummies(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pclass      1309 non-null   int64  
 1   survived    1309 non-null   int64  
 2   age         1046 non-null   float64
 3   sibsp       1309 non-null   int64  
 4   parch       1309 non-null   int64  
 5   fare        1308 non-null   float64
 6   sex_female  1309 non-null   bool   
 7   sex_male    1309 non-null   bool   
 8   embarked_C  1309 non-null   bool   
 9   embarked_Q  1309 non-null   bool   
 10  embarked_S  1309 non-null   bool   
dtypes: bool(5), float64(2), int64(4)
memory usage: 67.9 KB


Utiliza-se `pd.get_dummies` para criar colunas dummy para as colunas categóricas

Essas colunas dummy são binárias, ou seja, possuem apenas 2 valores, 0 ou 1


In [123]:
df = df.drop(columns='sex_male')

Remove-se a coluna sex_male, pois ela é inversa a coluna sex_female. Se o valor de uma for 0, o valor da outra será 1 e vice-versa. Esse tipo de relação é chamada de correlação perfeita negativa. Essas relações podem causar problemas no modelo, pois o modelo pode entender que uma coluna é mais importante que a outra, quando na verdade elas são iguais.

In [124]:
df = pd.get_dummies(df, drop_first=True)
df.columns

Index(['pclass', 'survived', 'age', 'sibsp', 'parch', 'fare', 'sex_female',
       'embarked_C', 'embarked_Q', 'embarked_S'],
      dtype='object')

Passando o parâmetro `drop_first=True` para `pd.get_dummies`, a primeira coluna dummy será removida, evitando assim a correlação perfeita negativa.

In [125]:
X = df.drop(columns='survived')
y = df.survived

```python
X = df.drop(columns='survived')
y = df['survived']
```

X é um dataframe com todas as colunas, exceto a coluna survived

y é uma série com a coluna survived


#### Separando as amostras

In [126]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
)

O método `train_test_split` separa os dados em amostras de treino e teste. O parâmetro `test_size` define a porcentagem de dados que serão separados para teste, o padrão é 25%. Para o exemplo está sendo utilizado 30%, o que quer dizer que 70% dos dados são para treino e 30% para teste.

O parametro `random_state` define a semente do gerador de números aleatórios, isso garante que a separação dos dados seja sempre a mesma, ou seja, sempre que o código for executado, os mesmos dados serão separados para treino e teste.

`X_train` é um dataframe contendo 70% dos dados de X

`X_test` é um dataframe contendo 30% dos dados de X

`y_train` é uma série contendo 70% dos dados de y

`y_test` é uma série contendo 30% dos dados de y


#### Faça a imputação dos dados

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pclass      1309 non-null   int64  
 1   survived    1309 non-null   int64  
 2   age         1046 non-null   float64
 3   sibsp       1309 non-null   int64  
 4   parch       1309 non-null   int64  
 5   fare        1308 non-null   float64
 6   sex_female  1309 non-null   bool   
 7   embarked_C  1309 non-null   bool   
 8   embarked_Q  1309 non-null   bool   
 9   embarked_S  1309 non-null   bool   
dtypes: bool(4), float64(2), int64(4)
memory usage: 66.6 KB


In [128]:
from sklearn import impute

In [129]:
num_cols = [
    'pclass',
    'age',
    'sibsp',
    'parch',
    'fare',
]

In [130]:
imputer = impute.IterativeImputer()

imputed = imputer.fit_transform(
    X_train[num_cols]
)
X_train.loc[:, num_cols] = imputed

imputed = imputer.transform(
    X_test[num_cols]
)
X_test.loc[:, num_cols] = imputed

nan_values_x_train = X_train.isnull().sum()
nan_values_x_test = X_test.isnull().sum()

print(f'O numero de valores nulos no X_train é: {nan_values_x_train.sum()}'\
      f'\nO numero de valores nulos no X_test é: {nan_values_x_test.sum()}')

O numero de valores nulos no X_train é: 0
O numero de valores nulos no X_test é: 0


Uma das formas de lidar com esses valores nulos é através da imputação, que é a substituição dos valores nulos por outros valores. Foi utilizada a biblioteca `sklearn.impute` para fazer a imputação.

Inicialmente cria-se uma variavel `num_cols`  com as colunas numéricas.

Em seguida cria-se `imputer` com o método `impute.IterativeImputer()`, ou seja `imputer` é um objeto da classe `IterativeImputer`.

Cria-se em sequência a variavel `imputed` através `imputer.fit_transform(X_tra[num_cols]`, ou seja, `imputed` é um array numpy com os valores imputados.

Em sequência é feita a indexação de `imputed` para `X_train[num_cols]` e `X_test[num_cols]`, ou seja, os valores imputados são atribuidos a `X_train[num_cols]` e `X_test[num_cols]`.



In [131]:
meds = X_train.median()

X_train = X_train.fillna(meds)
X_test = X_test.fillna(meds)

nan_values_x_train = X_train.isnull().sum()
nan_values_x_test = X_test.isnull().sum()

print(f'O numero de valores nulos no X_train é: {nan_values_x_train.sum()}'\
        f'\nO numero de valores nulos no X_test é: {nan_values_x_test.sum()}')

O numero de valores nulos no X_train é: 0
O numero de valores nulos no X_test é: 0


De modo semelhante pode-se imputar valores utilizando a mediana por exemplo.

Cria-se uma variavel chamada `meds`, ela recebe a mediana de `X_train`, e em sequência utiliza-se o método `fillna` para substituir os valores nulos de `X_train` e `X_test` pela mediana.

### Normalize os dados

In [132]:
from sklearn import preprocessing

cols = ['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex_female', 'embarked_C',
   'embarked_Q', 'embarked_S']
sca = preprocessing.StandardScaler()
X_train = sca.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns=cols)
X_test = sca.transform(X_test)
X_test = pd.DataFrame(X_test, columns=cols)



### Refatore

In [133]:
def tweaky_titanic (df):
    df.drop(
        columns=[
            'name',
            'ticket',
            'home.dest',
            'boat',
            'body',
            'cabin',
        ],
    ).pipe(pd.get_dummies, drop_first=True)
    return df

A função `.pipe` permite encadear funções, ou seja, o resultado de uma função é passado como parâmetro para a próxima função.

O parâmetro `drop_first=True` é passado para `pd.get_dummies` através de `.pipe`, evitando assim a correlação perfeita negativa.

Uma breve explicação sobre a função `tweaky_titanic` :

```python

def tweaky_titanic (df):
    df.drop(
        columns=[
            'name',
            'ticket',
            'home.dest',
            'boat',
            'body',
            'cabin',
        ],
    ).pipe(pd.get_dummies, drop_first=True)
    return df

```

A função irá receber um data frame como parâmetro, e em seguida irá remover as colunas name, ticket, home.dest, boat, body e cabin. Em seguida irá criar colunas dummy para as colunas categóricas, e por fim irá retornar o data frame.

In [134]:
from sklearn.experimental import enable_iterative_imputer
def get_train_test_X_y (
        df, y_col, size = 0.3, std_cols = None
):
    y = df[y_col]
    X = df.drop(columns=y_col)

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=size,
        random_state=42,
    )

    cols = X.columns
    num_cols = [
        'pclass',
        'age',
        'sibsp',
        'parch',
        'fare',
    ]
    
    fi = impute.IterativeImputer()
    X_train.loc[:, num_cols] = fi.fit_transform(
        X_train[num_cols]
    )

    X_test.loc[:, num_cols] = fi.transform(
        X_test[num_cols]
    )

    if std_cols:
        std = preprocessing.StandardScaler()
        X_train.loc[:, std_cols] = std.fit_transform(
            X_train[std_cols]
        )

        X_test.loc[:, std_cols] = std.transform(
            X_test[std_cols]
        )

    return X_train, X_test, y_train, y_test


Explicação sobre a função `get_train_text_X_y` :

```python
from sklearn.experimental import enable_iterative_imputer
def get_train_test_X_y (
        df, y_col, size = 0.3, std_cols = None
):
    y = df[y_col]
    X = df.drop(columns=y_col)

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=size,
        random_state=42,
    )

    cols = X.columns
    num_cols = [
        'pclass',
        'age',
        'sibsp',
        'parch',
        'fare',
    ]
    
    fi = impute.IterativeImputer()
    X_train.loc[:, num_cols] = fi.fit_transform(
        X_train[num_cols]
    )

    X_test.loc[:, num_cols] = fi.transform(
        X_test[num_cols]
    )

    if std_cols:
        std = preprocessing.StandardScaler()
        X_train.loc[:, std_cols] = std.fit_transform(
            X_train[std_cols]
        )

        X_test.loc[:, std_cols] = std.transform(
            X_test[std_cols]
        )

    return X_train, X_test, y_train, y_test

```

Inicia-se importando a classe `IterativeImputer` da biblioteca `sklearn.experimental`, essa classe permite fazer a imputação dos dados.

A função recebe como parâmetro um data frame, o nome da coluna alvo, o tamanho da amostra de teste, e uma lista com as colunas que serão padronizadas.

Em seguida é feita a separação dos dados em amostras de treino e teste.

Em seguida é criada uma lista com as colunas numéricas.

Em seguida é criado um objeto da classe `IterativeImputer` e em seguida é feita a imputação dos dados de treino e teste.

Em seguida é feita a padronização dos dados de treino e teste.

A padronização é feita através da classe `StandardScaler` da biblioteca `sklearn.preprocessing`. Ela so é feita se o parâmetro `std_cols` for diferente de `None`.

Por fim a função retorna os dados de treino e teste.


In [135]:
ti_df = tweaky_titanic(df_orig)

std_cols = "pclass, age, sibsp, parch, fare".split(", ")

X_train, X_test, y_train, y_test = get_train_test_X_y(
    ti_df, "survived", std_cols=std_cols
)

Explicação do código acima:

```python

ti_df = tweaky_titanic(df_orig)

std_cols = "pclass, age, sibsp, parch, fare".split(", ")

X_train, X_test, y_train, y_test = get_train_test_X_y(
    ti_df, "survived", std_cols=std_cols
)

```

A variavel `ti_df` recebe o resultado da função `tweaky_titanic` que recebe como parâmetro o data frame original.

A variavel `std_cols` recebe uma lista com as colunas que serão padronizadas.

Em seguida as variaveis `X_train, X_test, y_train, y_test` recebem o resultado da função `get_train_test_X_y` que recebe como parâmetro o data frame `ti_df`, o nome da coluna alvo, e a lista com as colunas que serão padronizadas.

```

In [136]:
from sklearn.dummy import DummyClassifier

bm = DummyClassifier()
bm.fit(X_train, y_train)
bm.score(X_test, y_test)

0.5699745547073791

Explicação do código acima:

```python

from sklearn.dummy import DummyClassifier

bm = DummyClassifier()
bm.fit(X_train, y_train)
bm.score(X_test, y_test)

```

A variavel `bm` recebe um objeto da classe `DummyClassifier` da biblioteca `sklearn.dummy`. Essa classe cria um modelo que faz previsões aleatórias.

Em seguida é feito o treinamento do modelo através do método `fit` e em seguida é feita a previsão através do método `score`.

O resultado é 0.55, ou seja, o modelo acerta 55% das previsões.

In [138]:
from sklearn import metrics
metrics.precision_score(
    y_test, bm.predict(X_test)
)

/home/pcmoraes/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0

Explicação do código acima:

```python

from sklearn import metrics
metrics.precision_score(
    y_test, bm.predict(X_test)
)

```

A função `precision_score` da biblioteca `sklearn.metrics` calcula a precisão do modelo, ou seja, a quantidade de previsões corretas dividido pela quantidade de previsões totais.

O resultado é 0.55, ou seja, o modelo acerta 55% das previsões.

